<a href="https://colab.research.google.com/github/ipeirotis/introduction-to-databases/blob/master/session5/practice_queries_join_and_aggregation_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 5: Practice Aggregate+Join Queries: Solutions


In this segment we will connect to the *Restaurants* database.

## Setup

In [0]:
!apt-get install python3-mysqldb
!pip install sqlalchemy sql_magic

In [0]:
# This code creates a connection to the database
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'restaurants',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)
con = engine.connect()

In [0]:
%reload_ext sql_magic

In [0]:
%config SQL.conn_name = 'engine'

## Question 1: Output the restaurant name together with the number of reviews submitted for this restaurant.

Let's run first the join query, and look at the table.

This will be the table on which the GROUP BY query will operate.

We order by `restName` to visually illustrate the groups that will be created.

In [0]:
%%read_sql
SELECT *
FROM Restaurant R INNER JOIN Rating T ON R.restCode=T.restCode
ORDER BY restName

Now notice the output when we use a LEFT JOIN instead of an INNER JOIN. Notice the extra restaurants that appear, which have received no reviews (and therefore the `code` and `cID` are NULL)

In [0]:
%%read_sql
SELECT *
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
ORDER BY restName

We now execute the GROUP BY. Notice how we use the `COUNT` command to count the total number of reviews, and the reviews with comments. Since this is an `INNER JOIN`, the `COUNT(*)` and `COUNT(code)` return the same values.

In [0]:
%%read_sql
SELECT restName, 
    COUNT(*) AS cnt, 
    COUNT(code) AS num_reviews, 
    COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R INNER JOIN Rating T ON R.restCode=T.restCode
GROUP BY restName

Now, let's switch to a LEFT JOIN. Notice an important change. We cannot rely on `COUNT(*)` anymore to count the number of reviews, and we need to be using the `COUNT(code)`. For example `Nisi` has one review: both the `cnt` and the `num_reviews` column are 1. However, for the `Don Peppe`, which has no reviews, we see that `cnt` is still 1, but `num_reviews` is 0.

In [0]:
%%read_sql
SELECT restName, 
    COUNT(*) AS cnt, 
    COUNT(code) AS num_reviews, 
    COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
GROUP BY restName

## Question 2: For every Manhattan restaurant output its name and the number of reviews submitted for this restaurant.

In [0]:
%%read_sql
SELECT restName, COUNT(code) AS num_reviews, COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
WHERE borough = 'Manhattan'
GROUP BY restName

## Question 3: For every restaurant that was reviewed more than once output it name and the number or reviews submitted for this restaurant.

In [0]:
%%read_sql
SELECT restName, COUNT(code) AS num_reviews, COUNT(comments) AS num_reviews_with_comments
FROM Restaurant R LEFT JOIN Rating T ON R.restCode=T.restCode
GROUP BY restName
HAVING num_reviews>1

## Question 4: Output the critic's last name and the restaurant name together with the maximal star rating ever issued by this critic for this restaurant.

Again, let's execute first the JOIN of all the tables that we need, so that we can see the data that we will be aggregating. Since we want to organize by critic's last name and restaurant name, we also add the corresponding ORDER BY, so that we can visually group together the rows that we will be aggregating.

In [0]:
%%read_sql
SELECT *
FROM Critic C 
  INNER JOIN Rating R ON R.cID = C.cID
  INNER JOIN Restaurant T ON T.restCode = R.restCode
ORDER BY restName, lastN

Now, let's report the number of reviews that a critic wrote for the restaurant, together with the mix and max ratings. Obviously, when there is only one review, the min and max ratings are the same.

In [0]:
%%read_sql
SELECT restName, lastN, 
    COUNT(R.code) AS num_reviews, 
    MAX(R.starRating) AS maxRating, 
    MIN(R.starRating) AS minRating 
FROM Critic C 
  INNER JOIN Rating R ON R.cID = C.cID
  INNER JOIN Restaurant T ON T.restCode = R.restCode
GROUP BY restName, lastN
ORDER BY restName, lastN

## Question 5: For each cuisine-borough pair, output the number of the corresponding restaurants.

In [0]:
%%read_sql
SELECT cuisine, borough, COUNT(*) AS num_restaurants
FROM Restaurant
GROUP BY cuisine, borough

## Question 6: For each NYT reporter, output the number of distinct restaurants this reporter reviewed.

In [0]:
%%read_sql
SELECT *
FROM Critic C
  INNER JOIN Rating R ON R.cID = C.cID
ORDER BY C.cID, R.restCode

Now let's execute the GROUP BY. Notice that we only GROUP by the `C.cID` which is the primary key for the table `Critic C`. Since the `cID` is the primary key for that table, we can also add in the SELECT clause the other attributes of the critic (which are unique for a given cID) without adding these attributes in the GROUP BY clause.

In [0]:
%%read_sql
SELECT C.cID, 
  firstN, lastN, affiliation, 
  COUNT(DISTINCT R.restCode) AS num_distinct_restaurants
FROM Critic C
  INNER JOIN Rating R ON R.cID = C.cID
GROUP BY C.cID

In [0]:
%%read_sql
SELECT C.cID, 
  firstN, lastN, affiliation, 
  COUNT(DISTINCT R.restCode) AS num_distinct_restaurants
FROM Critic C
  INNER JOIN Rating R ON R.cID = C.cID
WHERE affiliation = 'NYT'
GROUP BY C.cID

## Question 7: For every news outlet, output the average star rating submitted by all the reviewers of this outlet. a. consider only Italian restaurants b. consider only Italian restaurants outside of Manhattan

In [0]:
%%read_sql
SELECT *
FROM Critic C
  INNER JOIN Rating R ON R.cID = C.cID
ORDER BY C.affiliation

In [0]:
%%read_sql
SELECT C.affiliation, COUNT(*) AS num_reviews, AVG(starRating) AS avgRating
FROM Critic C
  INNER JOIN Rating R ON R.cID = C.cID
GROUP BY C.affiliation
ORDER BY C.affiliation

## Question 8: For every borough output the max star rating submitted for any restaurant within this borough (in which borough do you have the best restaurant)

In [0]:
%%read_sql
SELECT *
FROM Restaurant R 
  INNER JOIN Rating T ON T.restCode = R.restCode
ORDER BY borough

In [0]:
%%read_sql
SELECT borough, MAX(starRating) AS maxRating
FROM Restaurant R 
  INNER JOIN Rating T ON T.restCode = R.restCode
GROUP BY borough
ORDER BY borough

In [0]:
%%read_sql
SELECT *
FROM Restaurant R 
  LEFT JOIN Rating T ON T.restCode = R.restCode
ORDER BY borough

In [0]:
%%read_sql
SELECT borough, MAX(starRating) AS maxRating
FROM Restaurant R 
  LEFT JOIN Rating T ON T.restCode = R.restCode
GROUP BY borough
ORDER BY borough